In [1]:
import sys
sys.path.append("/home/administrator/Documenti/APDFT/prototyping/hessian/AAFF/Libs/")
import inspect
from aaff import aaff,alc_deriv_grad_nuc,alc_differential_grad_nuc
from FcMole import FcM,FracMole,FcM_like
from pyscf import gto,scf
import numpy as np 
from numpy.linalg import norm as norm
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
from pyscf.grad import rhf as grhf
from pyscf.hessian import rhf as hrhf
import basis_set_exchange as bse
from FDcoeffs import *
from pyscf.geomopt.berny_solver import optimize
ang2bohr=1.8897261246
bohr2ang=.5291772109
%load_ext autoreload
%autoreload 2
from AP_class import benz_Symm,APDFT_perturbator,alch_deriv,first_deriv_nuc_nuc,second_deriv_nuc_nuc,DeltaV,alchemy_cphf_deriv
from AP_class import first_deriv_elec,second_deriv_elec,third_deriv_elec
import AP_class as apc
np.set_printoptions(precision=4)

In [2]:
benz_atom=""" 
C         6.95191249e-13  2.61173060e+00 -1.67580194e-08 ;
H         2.83052087e-13  4.63941350e+00 -1.15976219e-08 ;
C         2.26182505e+00  1.30586530e+00 -2.37508246e-08 ; 
H         4.01784995e+00  2.31970675e+00 -9.84687205e-09 ; 
C         2.26182505e+00 -1.30586530e+00  8.26065094e-09 ; 
H         4.01784995e+00 -2.31970675e+00  5.83890508e-08 ; 
C        -8.45954456e-13 -2.61173060e+00 -1.55922267e-08 ; 
H         1.49445304e-12 -4.63941350e+00 -3.26774154e-08 ; 
C        -2.26182505e+00 -1.30586530e+00 -8.28266973e-09 ; 
H        -4.01784995e+00 -2.31970675e+00  1.87236306e-09 ; 
C        -2.26182505e+00  1.30586530e+00  1.68546675e-08 ; 
H        -4.01784995e+00  2.31970675e+00  3.31289175e-08 ; 
"""
Benz = gto.M(atom=benz_atom,unit='bohr',basis='sto-3g',symmetry = 'D6v') #basis={"H":"pc-2",'C':bse.get_basis("pcX-2",fmt="nwchem",elements=[6])})

In [3]:
BenzHF=scf.RHF(Benz)
BenzHF.scf()

converged SCF energy = -227.890711338273


-227.89071133827343

In [4]:
d1,d2,d3= alch_deriv(BenzHF,[[0],[1]])
d1,d2,d3

(-14.554808443274133, -1.079627074618511, 0.26021645259666804)

In [5]:
BenzHF.energy_elec(),BenzHF.e_tot,BenzHF.energy_nuc()

((-433.2936110314304, 282.18849680677613),
 -227.89071133827343,
 205.40289969315694)

In [6]:
dl=1e-3
e0=BenzHF.energy_elec()[0]
e0n=BenzHF.energy_nuc()
fdhf1=scf.RHF(FcM(fcs=[[0,1],[dl,0]],atom=benz_atom,unit='bohr',basis='sto-3g',verbose=1))
fdhf1.scf()
e1=fdhf1.energy_elec()[0]
e1n=fdhf1.energy_nuc()
fdhf2=scf.RHF(FcM(fcs=[[0,1],[0,dl]],atom=benz_atom,unit='bohr',basis='sto-3g',verbose=1))
fdhf2.scf()
e2=fdhf2.energy_elec()[0]
e2n=fdhf2.energy_nuc()

In [7]:
fdhf3=scf.RHF(FcM(fcs=[[0,1],[dl,dl]],atom=benz_atom,unit='bohr',basis='sto-3g',verbose=1))
fdhf3.scf()
e3=fdhf3.energy_elec()[0]
e3n=fdhf3.energy_nuc()
fdhf4=scf.RHF(FcM(fcs=[[0,1],[-2*dl,2*dl]],atom=benz_atom,unit='bohr',basis='sto-3g',verbose=1))
fdhf4.scf()
e4=fdhf4.energy_elec()[0]
e4n=fdhf4.energy_nuc()

In [8]:
((e3-e2)-(e1-e0))/dl**2 ,((e3n-e2n)-(e1n-e0n))/dl**2# = d/dZ3(dE/dZ0 )

(-0.01704063379293075, 0.4931737294100458)

In [9]:
second_deriv_nuc_nuc(Benz,[[0,3],[1,1]])

0.4964690679154011

In [10]:
((e3-e2)-(e1-e0))/dl**2 # = d/dZ3(dE/dZ0 )

-0.01704063379293075

In [11]:
mo1=alchemy_cphf_deriv(BenzHF,DeltaV(Benz,[[3],[1]]))[0]
second_deriv_elec(BenzHF,DeltaV(Benz,[[0],[1]]),mo1)

0.0658700665306905

In [12]:
((e3n-e1n)-(e2n-e0n))/dl**2 #= d/dZ0(dE/dZ3 )

0.4931737294100458

In [13]:
second_deriv_nuc_nuc(Benz,[[0,3],[1,1]])

0.4964690679154011

In [14]:
d1,d2,d3= alch_deriv(BenzHF,[[0],[1]])
d1,d2,d3

(-14.554808443274133, -1.0796270746185113, 0.2602164525966687)

In [15]:
AP=APDFT_perturbator(BenzHF,sites=[x for x in range(12)])

In [16]:
AP.build_gradient(*AP.sites),AP.build_hessian(*AP.sites),AP.build_cubic_hessian(*AP.sites)

(array([-14.5548,  -1.1429, -14.5548,  -1.1429, -14.5548,  -1.1429,
        -14.5548,  -1.1429, -14.5548,  -1.1429, -14.5548,  -1.1429]),
 array([[-1.0796,  0.4762,  0.4004,  0.3141,  0.2636,  0.2485,  0.296 ,
          0.2632,  0.2636,  0.2485,  0.4004,  0.3141],
        [ 0.4762, -0.7568,  0.3141,  0.2854,  0.2485,  0.2321,  0.2632,
          0.2374,  0.2485,  0.2321,  0.3141,  0.2854],
        [ 0.4004,  0.3141, -1.0796,  0.4762,  0.4004,  0.3141,  0.2636,
          0.2485,  0.296 ,  0.2632,  0.2636,  0.2485],
        [ 0.3141,  0.2854,  0.4762, -0.7568,  0.3141,  0.2854,  0.2485,
          0.2321,  0.2632,  0.2374,  0.2485,  0.2321],
        [ 0.2636,  0.2485,  0.4004,  0.3141, -1.0796,  0.4762,  0.4004,
          0.3141,  0.2636,  0.2485,  0.296 ,  0.2632],
        [ 0.2485,  0.2321,  0.3141,  0.2854,  0.4762, -0.7568,  0.3141,
          0.2854,  0.2485,  0.2321,  0.2632,  0.2374],
        [ 0.296 ,  0.2632,  0.2636,  0.2485,  0.4004,  0.3141, -1.0796,
          0.4762,  0.4004,  

In [17]:
b2p=FcM_like(Benz,[[0,2],[-1,1]])

In [18]:
b2mf=scf.RHF(b2p)

In [19]:
b2mf.scf()

converged SCF energy = -229.396518336255


/home/administrator/anaconda3/lib/python3.7/site-packages/pyscf/gto/mole.py:1072: UserWarning: Function mol.dumps drops attribute with_rinv_at_nucleus because it is not JSON-serializable
  warnings.warn(msg)


-229.3965183362555

In [20]:
b2mf.max_cycle=50 
b2mf.scf(dm0=b2mf.init_guess_by_1e())

converged SCF energy = -229.396518336226


-229.3965183362256

In [21]:
def targ_energy(pvec):
    b2p=FcM_like(Benz,pvec.tolist())
    b2mf=scf.RHF(b2p)
    return b2mf.scf()

In [22]:
pvec21=apc.parse_to_array(12,[[0,2],[1,-1]])
pvec22=apc.parse_to_array(12,[[0,3],[1,-1]])
pvec23=apc.parse_to_array(12,[[0,6],[1,0]])

In [23]:
AP.APDFT1(pvec21),AP.APDFT2(pvec21),AP.APDFT3(pvec21),AP.targ_energy(pvec21)

converged SCF energy = -229.396518336244


(-227.89071133829376,
 -229.37070393795392,
 -229.3707039379321,
 -229.39651833624382)

In [24]:
AP.APDFT1(pvec22),AP.APDFT2(pvec22),AP.APDFT3(pvec22),targ_energy(pvec22)

converged SCF energy = -242.434488202582


(-241.30257728376176,
 -242.53490561942365,
 -242.46417806683507,
 -242.43448820258175)

In [25]:
AP.APDFT1(pvec23),AP.APDFT2(pvec23),AP.APDFT3(pvec23),targ_energy(pvec23)

converged SCF energy = -242.951354669314


(-242.44551978154757,
 -242.98533331885682,
 -242.9419639100907,
 -242.95135466931353)

In [26]:
d1,d2,d3= alch_deriv(BenzHF,pvec21.tolist())
d1,d2/2,d3/6,AP.mf.e_tot+d1,AP.mf.e_tot+d1+.5*d2,AP.mf.e_tot+d1+d2*.5+1/6*d3

(-2.0364890329638285e-11,
 -1.479992600277654,
 2.1886329554243744e-11,
 -227.8907113382938,
 -229.37070393857147,
 -229.37070393854958)

In [27]:
import inspect
from berny import Berny, geomlib, coords,Geometry

In [28]:
m21=AP.targ_mol(pvec21)

In [29]:
m21.atom_charges()

array([7., 1., 5., 1., 6., 1., 6., 1., 6., 1., 6., 1.])

In [30]:
charge2symbol={1:"H",2:"He",3:"Li",4:"Be",5:"B",6:"C",7:"N",8:"O",9:"F",10:"Ne"}

charge2symbol[m21.atom_charge(2)]

'B'

In [31]:
charge2symbol={1:"H",2:"He",3:"Li",4:"Be",5:"B",6:"C",7:"N",8:"O",9:"F",10:"Ne"}
pbg=Geometry([charge2symbol[x] for x in m21.atom_charges()],m21.atom_coords()*bohr2ang)

In [32]:
b=Berny(pbg,debug=True)

In [33]:
b.send((AP.APDFT3(pvec21),AP.af(0)-AP.af(2)))  #  fcs=[0,2],[1,-1]

No AF found for atom 0. Calculating it from code


{'geom': <Geometry 'BC4H6N'>,
 'params': {'gradientmax': 0.00045,
  'gradientrms': 0.00015,
  'stepmax': 0.0018,
  'steprms': 0.0012,
  'trust': 0.3,
  'dihedral': True,
  'superweakdih': False},
 'trust': 0.3,
 'coords': <InternalCoords "bonds: 12, angles: 18, dihedrals: 24">,
 'H': array([[0.4733, 0.    , 0.    , ..., 0.    , 0.    , 0.    ],
        [0.    , 0.5072, 0.    , ..., 0.    , 0.    , 0.    ],
        [0.    , 0.    , 0.4927, ..., 0.    , 0.    , 0.    ],
        ...,
        [0.    , 0.    , 0.    , ..., 0.0066, 0.    , 0.    ],
        [0.    , 0.    , 0.    , ..., 0.    , 0.0065, 0.    ],
        [0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.0063]]),
 'weights': array([1.0517, 1.1272, 1.095 , 1.1116, 1.1397, 1.0692, 1.108 , 1.0692,
        1.108 , 1.0692, 1.108 , 1.0692, 0.9604, 0.9466, 0.98  , 0.9874,
        0.9998, 0.9929, 0.9738, 0.9913, 0.9601, 0.9601, 0.9774, 0.9601,
        0.9601, 0.9774, 0.9601, 0.9716, 0.9545, 0.9601, 0.8531, 0.8602,
        0.8646, 0.8718, 

In [42]:
b._state.previous.g.shape  #len=54 = 12 bonds, 18 angles, 24 dihedrals 

(54,)

In [73]:
%time AP.APDFT3(pvec23)

CPU times: user 162 µs, sys: 9 µs, total: 171 µs
Wall time: 220 µs


-242.9419639100908

In [96]:
dir(b)

['Point',
 'State',
 '__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__next__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_converged',
 '_debug',
 '_log',
 '_log_extra',
 '_maxsteps',
 '_n',
 '_state',
 'close',
 'converged',
 'send',
 'throw',
 'trust']

In [100]:
b.State

berny.berny.Berny.State

In [83]:
b.Point.__repr__(b.Point)

TypeError: not enough arguments for format string

In [88]:
b.Point.__getitem__('E')

TypeError: descriptor '__getitem__' requires a 'tuple' object but received a 'str'

In [81]:
help(b.Point)

Help on class Point in module berny.berny:

class Point(builtins.tuple)
 |  Point(q, E, g)
 |  
 |  Point(q, E, g)
 |  
 |  Method resolution order:
 |      Point
 |      builtins.tuple
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __getnewargs__(self)
 |      Return self as a plain tuple.  Used by copy and pickle.
 |  
 |  __repr__(self)
 |      Return a nicely formatted representation string
 |  
 |  _asdict(self)
 |      Return a new OrderedDict which maps field names to their values.
 |  
 |  _replace(_self, **kwds)
 |      Return a new Point object replacing specified fields with new values
 |  
 |  ----------------------------------------------------------------------
 |  Class methods defined here:
 |  
 |  _make(iterable) from builtins.type
 |      Make a new Point object from a sequence or iterable
 |  
 |  ----------------------------------------------------------------------
 |  Static methods defined here:
 |  
 |  __new__(_cls, q, E, g)
 |      Create new